<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2023notebooks/2023_0417random_forest_HAD_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* sepal (萼片 がくべん)： 花の外側の部分 (多くの場合緑色で葉のようなもの) で発達中の蕾を包む。
* petal (花弁，花びら)：花の外側の部分で，しばしば目立つ色をしている。

<img src="https://dictionary.goo.ne.jp/img/daijisen/ref/113205.jpg"><br/>
Goo 国語辞典より



In [1]:
# 必要なライブラリを輸入
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

In [ ]:
# HAD データの取得
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade xlrd
    !pip install --upgrade pandas

    # HAD サンプルデータをダウンロード
    !wget 'https://files.au-1.osf.io/v1/resources/32cyp/providers/osfstorage/5fb340145502ac018d8c86ab/?zip=' -O had.zip
    !unzip had.zip

# HAD のサンプルデータ読み込み
df = pd.read_excel('HAD_sample_data.xls', sheet_name="iris", index_col='ID')
df

In [3]:
# 説明変数を X とする
X = df.drop('Species', axis=1)

# 被説明変数を y とする
y = df['Species']

In [ ]:
# データセットを訓練データとテストデータに分割
from sklearn.model_selection import train_test_split

# ランダムにデータを訓練用と検証用に振り分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(f'訓練データ:{X_train.shape}, 検証データ:{X_test.shape}')

In [5]:
# ランダムフォレスト分類器を輸入して，定義
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(random_state=42, 
                                       n_jobs=-1, 
                                       max_depth=5,
                                       n_estimators=100, 
                                       criterion="entropy",  # ['gini', 'entropy']
                                       oob_score=True)

# oob_score: OOB は out of the bag の意味。
# ランダムフォレストの交差検証法の一つ。
# この方法では，サンプルの 3 分の 1 はデータの訓練に使用されず，代わりに性能の評価に使用される。
# これらのサンプルは out-of-bag サンプルと呼ばれる。

In [ ]:
%%time
classifier_rf.fit(X_train, y_train)

# oob 得点の出力
print(f'oob 得点:{classifier_rf.oob_score_:.3f}')

In [12]:
# `GridSearchCV` を用いたランダムフォレストのハイパーパラメータチューニング
rf = RandomForestClassifier(random_state=42, 
                            n_jobs=-1,
                            criterion='entropy',
                           )
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

from sklearn.model_selection import GridSearchCV
# グリッド探索モデルの実体化
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=1, 
                           #n_jobs=-1, 
                           verbose=1, 
                           scoring="accuracy")

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
print(f'グリッドサーチによる最良モデルの得点:{grid_search.best_score_:.3f}')

In [ ]:
rf_best = grid_search.best_estimator_
rf_best

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(10,4))
#plt.figure(figsize=(80,40))
plot_tree(rf_best.estimators_[3], 
          feature_names=X.columns,
          class_names=['1','2','3'],
          filled=True)

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(8,4))
#plt.figure(figsize=(80,40))
plot_tree(rf_best.estimators_[7], 
          feature_names = X.columns,
          class_names=['1','2','3'],
          filled=True)

In [ ]:
print(rf_best.feature_importances_)

imp_df = pd.DataFrame({
    "変数": X_train.columns,
    "重要度": rf_best.feature_importances_
})
imp_df.sort_values(by="重要度", ascending=False)